In [1]:
import neo4j
import osmnx as ox
import pandas as pd

In [2]:
NEO4J_URI = "bolt://localhost:7689"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "busticket123"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [3]:
# Define the area of interest
place = "Hamburg, Deutschland"

# Define the tags for different POIs you're interested in
tags = {
    # Additional categories
    'building': ['civic', 'public'],
}

In [6]:
# Fetch the POIs
pois = ox.features_from_place(place, tags=tags)

In [8]:
# description based on: 'description:de', 'description' 
# tags based on: office, amenity, community_centre, operator, building, sport, dsa:criteria, official_name, operator:type, tourism
impcols = ['geometry', 'name', 'addr:street', 'addr:housenumber', 'addr:postcode', 'description:de', 'description', 
          'office', 'amenity', 'community_centre', 'operator', 'building', 'sport', 'dsa:criteria', 'official_name', 'operator:type', 'tourism']

In [9]:
pois_impcols = pois[impcols]

In [10]:
pois_impcols.head()

geometry  \
element  id                                                               
node     1017868272                            POINT (9.97694 53.55304)   
         2737928311                            POINT (9.95034 53.56575)   
         6960079424                           POINT (10.07123 53.61907)   
         11012804769                          POINT (10.13497 53.62065)   
relation 308234       POLYGON ((9.98443 53.55664, 9.9844 53.55666, 9...   

                                               name   addr:street  \
element  id                                                         
node     1017868272         Handwerkskammer Hamburg   Holstenwall   
         2737928311             Spielhaus Alsenpark   Alsenstraße   
         6960079424   Bewegungsinsel Bramfelder See           NaN   
         11012804769     Elternschule Farmsen-Berne  Bekassinenau   
relation 308234                       Justizbehörde           NaN   

                     addr:housenumber addr:postcode  \
element  id                                           
node     1017868272                12         20355   
         2737928311                33         22769   
         6960079424               NaN           NaN   
         11012804769              126         22147   
relation 308234                   NaN           NaN   

                                                         description:de  \
element  id                                                               
node     1017868272                                                 NaN   
         2737928311   Offene Freizeiteinrichtung für Kinder/ Offene ...   
         6960079424                                                 NaN   
         11012804769                                                NaN   
relation 308234                                                     NaN   

                                                            description  \
element  id                                                               
node     1017868272                                                 NaN   
         2737928311                                                 NaN   
         6960079424                                                 NaN   
         11012804769  Familienförderung, Erwachsenenbildung, Offener...   
relation 308234                                                     NaN   

                          office           amenity community_centre  \
element  id                                                           
node     1017868272   government               NaN              NaN   
         2737928311          NaN  community_centre     youth_centre   
         6960079424          NaN               NaN              NaN   
         11012804769         NaN  community_centre              NaN   
relation 308234       government               NaN              NaN   

                                 operator building       sport dsa:criteria  \
element  id                                                                   
node     1017868272                   NaN   public         NaN          NaN   
         2737928311     Bezirksamt Altona    civic         NaN          NaN   
         6960079424                   NaN   public  gymnastics          NaN   
         11012804769  Bezirksamt Wandsbek   public         NaN          NaN   
relation 308234                       NaN   public         NaN          NaN   

                     official_name operator:type tourism  
element  id                                               
node     1017868272            NaN           NaN     NaN  
         2737928311            NaN           NaN     NaN  
         6960079424            NaN           NaN     NaN  
         11012804769           NaN           NaN     NaN  
relation 308234                NaN           NaN     NaN

In [39]:
# Funktion, um zu entscheiden, ob ein Polygon durch seinen Centroid ersetzt werden soll
def simplify_to_centroid_if_small(geometry, max_area_threshold=0.0000006791):  # Schwellenwert anpassen
    """
    Ersetzt ein Polygon durch seinen Centroid, wenn die Fläche unter dem Schwellenwert liegt.
    Der Schwellenwert ist in Quadratgrad (für WGS84). 
    0.0001 entspricht ungefähr 1 km² in Hamburg.
    0.0000679123 entspricht ungefähr 0,5 km² in Hamburg.
    0.0000006791 entspricht ungefähr 5.000 m² in Hamburg.
    """
    if geometry.geom_type in ['Polygon', 'MultiPolygon']:
        area = geometry.area
        if area < max_area_threshold:
            return geometry.centroid
    return None

# Anwenden auf deine POIs DataFrame
pois_polygons['simplified_geometry'] = pois_polygons['geometry'].apply(simplify_to_centroid_if_small)

c:\Users\paul-\anaconda3\envs\py310\lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [40]:
pois_polygons[['geometry', 'simplified_geometry', 'name', 'addr:street']].head()

geometry  \
element  id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
relation 308234                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [43]:
pois_polygons[pois_polygons.simplified_geometry == None][['geometry', 'simplified_geometry', 'name', 'addr:street']].head()

geometry  \
element  id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
relation 538432    POLYGON ((9.97664 53.55599, 9.97669 53.55597, 9.97671 53.55598, 9.97687 53.55591, 9.97686 53.5559, 9.97691 53.55588, 9.9769 53.55587, 9.97724 53.55572, 9.97727 53.55575, 9.9773 53.55573, 9.97731 53.55574, 9.97748 53.55566, 9.97747 53.55566, 9.97752 53.55564, 9.97741 53.55555, 9.9774 53.55555, 9.97718 53.55537, 9.9772 53.55536, 9.97708 53.55526, 9.97706 53.55526, 9.97684 53.55508, 9.97684 53.55508, 9.97672 53.55498, 9.97653 53.55506, 9.97649 53.55503, 9.97644 53.555, 9.97651 53.55497, 9.97652 53.55494, 9.97626 53.55472, 9.97623 53.55474, 9.97621 53.55472, 9.97615 53.55474, 9.97605 53.55475, 9.97595 53.55475, 9.97577 53.55478, 9.97573 53.55479, 9.97565 53.55481, 9.9756 53.55483, 9.97557 53.55483, 9.97542 53.5549, 9.9753 53.55498, 9.97517 53.55508, 9.9751 53.55518, 9.97504 53.55521, 9.97506 53.55522, 9.97503 53.55523, 9.9751 53.55529, 9.97509 53.5553, 9.9753 53.55546, 9.97533 53.55546, 9.97539 53.55543, 9.97544 53.55547, 9.97547 53.5555, 9.97528 53.55559, 9.9754 53.55568, 9.9754 53.55568, 9.97562 53.55586, 9.9756 53.55587, 9.97566 53.55593, 9.97572 53.55598, 9.97574 53.55597, 9.9759

In [42]:
pois.columns.to_list()

['geometry',
 'addr:city',
 'addr:housenumber',
 'addr:postcode',
 'addr:street',
 'building',
 'name',
 'office',
 'phone',
 'website',
 'wheelchair',
 'wheelchair:description',
 'wikidata',
 'addr:country',
 'amenity',
 'check_date',
 'check_date:opening_hours',
 'community_centre',
 'community_centre:for',
 'description:de',
 'opening_hours',
 'operator',
 'social_facility',
 'social_facility:for',
 'fee',
 'leisure',
 'sport',
 'description',
 'short_name',
 'surveillance',
 'wikipedia',
 'building:levels',
 'old_name',
 'roof:shape',
 'roof:levels',
 'source',
 'note',
 'architect',
 'construction_date',
 'dsa:criteria',
 'heritage',
 'heritage:operator',
 'historic',
 'image',
 'ref:dsa',
 'contact:website',
 'opening_hours:signed',
 'opening_hours:url',
 'alt_name',
 'contact:email',
 'contact:fax',
 'contact:phone',
 'layer',
 'operator:wikidata',
 'operator:wikipedia',
 'police',
 'official_name',
 'height',
 'emergency',
 'ref',
 'short_name:de',
 'government',
 'operator:typ